# RNN

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from SR import *
from zigzag import *
import matplotlib.pyplot as plt
import mplfinance as mpf
from random import sample
import ta
import os

In [2]:
#cd data/gbpjpy

C:\Users\kush.patel\Desktop\DS\RNN_tabular\data\gbpjpy


# Load Data


MAKE ALL THE DATA

In [11]:
files = sorted([i for i in os.listdir() if i[:3] == "DAT"])

df = pd.DataFrame(columns=["open", "high", "low", "close", "volume"])
for i in files:
    temp = pd.read_csv(i, names=["open", "high", "low", "close", "volume"])
    df = pd.concat([df, temp])

# fix index
df.index = pd.to_datetime([df.index[i][0] + " " + df.index[i][1] for i, _ in enumerate(df.index)])
print("index done")

# # change timeframe
logic = {'open'  : 'first',
         'high'  : 'max',
         'low'   : 'min',
         'close' : 'last',
         'volume': 'sum'}
offset = pd.offsets.timedelta(days=-6)

#df = pd.read_clipboard(parse_dates=['Date'], index_col=['Date'])
df = df.resample('W', loffset=offset).apply(logic)
print("resample done")
# add all technical analysis features and get singals
ta.add_momentum_ta(df, "high", "low", "close", "volume", fillna=True)
ta.add_others_ta(df, "close", fillna=True)
ta.add_trend_ta(df, "high", "low", "close", fillna=True)
ta.add_volatility_ta(df, "high", "low", "close", fillna=True)
print("ta done")
zigzag(df)[0].to_csv("gbpjpy_W.csv")
print("zigzag done")

index done
resample done
ta done
zigzag done


In [6]:
pd.read_csv("gbpjpy_M15.csv")

,index,open,high,low,close,volume,momentum_rsi,momentum_tsi,momentum_uo,momentum_stoch,...,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,1
0,2010-12-27 17:00:00,126.420,126.590,126.380,126.570,0,100.000000,0.000000,0.000000,90.476190,...,126.513333,126.723333,126.303333,0.331981,0.634921,0.0,0.0,0.165916,0.904762,NaN
1,2010-12-27 17:15:00,126.590,126.590,126.470,126.480,0,0.000000,-100.000000,3.030303,47.619048,...,126.513333,126.678333,126.348333,0.260842,0.398990,0.0,0.0,0.165975,0.476190,NaN
2,2010-12-27 17:30:00,126.500,126.500,126.140,126.250,0,0.000000,-100.000000,17.391304,24.444444,...,126.441111,126.671111,126.211111,0.363806,0.084541,0.0,0.0,0.355919,0.244444,NaN
3,2010-12-27 17:45:00,126.240,126.240,125.820,125.870,0,0.000000,-100.000000,15.178571,6.493506,...,126.325000,126.602500,126.047500,0.439343,-0.319820,0.0,1.0,0.609696,0.064935,NaN
4,2010-12-27 18:00:00,125.920,126.290,125.920,126.280,0,39.681291,-91.238549,37.662338,59.740260,...,126.292667,126.588667,125.996667,0.468752,0.478604,0.0,0.0,0.609708,0.597403,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343195,2020-10-10 15:45:00,136.229,136.306,136.198,136.255,0,55.905306,8.766467,55.110653,81.040892,...,136.191733,136.278033,136.105433,0.126733,0.866551,0.0,0.0,0.270897,0.590786,NaN
343196,2020-10-10 16:00:00,136.252,136.253,136.177,136.208,0,52.720337,8.331264,52.082076,63.568773,...,136.197933,136.280433,136.115433,0.121147,0.561010,0.0,0.0,0.270901,0.463415,NaN
343197,2020-10-10 16:15:00,136.209,136.232,136.176,136.178,0,50.733532,7.432242,45.319053,52.416357,...,136.209300,136.285800,136.132800,0.112327,0.295425,0.0,0.0,0.270899,0.382114,NaN
343198,2020-10-10 16:30:00,136.176,136.197,136.085,136.095,0,45.612022,5.187733,35.799923,21.561338,...,136.210067,136.285667,136.134467,0.111005,-0.261023,0.0,1.0,0.270912,0.157182,NaN


# Testing Cells

In [ ]:
%time
lookback = 432
n_candles = 50

# get signals (as a column in df) and zigzag lines
df_zz, zz = zigzag(df, 1, 10)

df_sr = sr(df_zz,lookback)

# use this df to only show the last few candles
df_sr_image = df_sr[-n_candles:].copy()

# create a subplot for each level
apdict = [mpf.make_addplot(df_sr_image[c], color="w", width=5, alpha=0.5) for c in df_sr_image.columns[8:] if (c > df_sr_image[-n_candles:].low.min()*0.995 and int(c) < df_sr_image[-n_candles:].high.max()*1.005)]

# custom colors (mainly to make the background black)
# Create my own `marketcolors` to use with the `nightclouds` style:
mc = mpf.make_marketcolors(up='g',down='r',inherit=True)

# Create a new style based on `nightclouds` but with my own `marketcolors`:
s  = mpf.make_mpf_style(base_mpf_style='nightclouds',marketcolors=mc)

# plot level
mpf.plot(df_sr_image, type="candle", style=s, axisoff=True, addplot=apdict, mav=25, volume=True, figsize=(10,10))